# 📈 Classical Time Series Analysis

**Phase 3: Temporal Data & RNN - Step 1**

---

## 🎯 Цели ноутбука

1. **Понять основы временных рядов:** стационарность, тренд, сезонность
2. **Классические модели:** ARIMA, SARIMA, Prophet
3. **Сравнить подходы** и понять, когда какой метод работает лучше
4. **Подготовка к Deep Learning:** RNN/LSTM будут в следующих шагах

---

## 📊 Датасет: Airline Passengers

**Контекст:** Ежемесячное количество пассажиров авиалиний (1949-1960).

**Почему этот датасет?**
- 📊 **Классический пример** для обучения временным рядам
- 📈 **Четкий тренд:** рост во времени
- 🔄 **Сезонность:** годичные циклы
- 🧪 **Нестационарность:** требует преобразований

**Методы:**
1. **ARIMA:** AutoRegressive Integrated Moving Average
2. **SARIMA:** Seasonal ARIMA для учета сезонности
3. **Prophet:** Автоматическая модель от Facebook

---

## 📚 Часть 1: Теория временных рядов

### 1.1 Что такое временной ряд?

**Временной ряд** — последовательность наблюдений $\{y_1, y_2, ..., y_T\}$, упорядоченных во времени.

**Ключевое отличие от табличных данных:**
- ✅ **Порядок важен** (нельзя перемешивать строки)
- ✅ **Временные зависимости** (прошлое влияет на будущее)
- ✅ **Автокорреляция** (значения коррелируют с собой в прошлом)

---

### 1.2 Компоненты временного ряда

Временной ряд можно разложить на компоненты:

$$y_t = T_t + S_t + R_t$$

где:
- $T_t$ — **Тренд** (долгосрочное направление)
- $S_t$ — **Сезонность** (повторяющиеся паттерны)
- $R_t$ — **Остатки** (случайный шум)

**Альтернативно (мультипликативная модель):**

$$y_t = T_t \times S_t \times R_t$$

---

### 1.3 Стационарность

**Стационарный временной ряд** имеет:
1. **Постоянное среднее:** $E[y_t] = \mu$ для всех $t$
2. **Постоянная дисперсия:** $Var(y_t) = \sigma^2$ для всех $t$
3. **Автокорреляция зависит только от лага:** $Cov(y_t, y_{t-k})$ зависит от $k$, но не от $t$

**Почему важна?** Большинство моделей (ARIMA) требуют стационарности.

**Как проверить?**
- **Визуально:** график ряда
- **ADF test (Augmented Dickey-Fuller):** $H_0$: ряд нестационарный
- **KPSS test:** $H_0$: ряд стационарный

**Как сделать стационарным?**
- **Differencing:** $y'_t = y_t - y_{t-1}$ (устраняет тренд)
- **Log transformation:** $y'_t = \log(y_t)$ (стабилизирует дисперсию)
- **Seasonal differencing:** $y'_t = y_t - y_{t-s}$ (устраняет сезонность)

---

### 1.4 ARIMA модель

**ARIMA(p, d, q)** = **AR**(p) + **I**(d) + **MA**(q)

#### AR (AutoRegressive) - авторегрессия

Текущее значение зависит от прошлых значений:

$$y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} + ... + \phi_p y_{t-p} + \varepsilon_t$$

- $p$ — порядок AR (сколько прошлых значений используем)
- $\phi_i$ — коэффициенты

**Пример AR(1):** $y_t = 0.8 y_{t-1} + \varepsilon_t$ (80% от прошлого + шум)

---

#### I (Integrated) - интегрирование

Сколько раз нужно применить differencing для стационарности:

$$y'_t = y_t - y_{t-1}$$

- $d=0$ — ряд уже стационарный
- $d=1$ — берем первую разность
- $d=2$ — вторая разность (редко)

---

#### MA (Moving Average) - скользящее среднее

Текущее значение зависит от прошлых ошибок:

$$y_t = \mu + \varepsilon_t + \theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + ... + \theta_q \varepsilon_{t-q}$$

- $q$ — порядок MA
- $\theta_i$ — коэффициенты

**Пример MA(1):** $y_t = \varepsilon_t + 0.5 \varepsilon_{t-1}$

---

#### Полная ARIMA(p, d, q)

$$
\left(1 - \sum_{i=1}^{p} \phi_i L^i \right) (1-L)^d y_t = \left(1 + \sum_{i=1}^{q} \theta_i L^i \right) \varepsilon_t
$$

где $L$ — lag operator: $L y_t = y_{t-1}$

**Как выбрать p, d, q?**
1. **d:** ADF test (пока ряд не станет стационарным)
2. **p:** ACF plot (автокорреляционная функция)
3. **q:** PACF plot (частная автокорреляция)
4. **Auto ARIMA:** автоматический перебор с AIC/BIC

---

### 1.5 SARIMA - Seasonal ARIMA

**SARIMA(p, d, q)(P, D, Q, s)** добавляет сезонные компоненты:

- **(p, d, q):** обычные ARIMA параметры
- **(P, D, Q, s):** сезонные параметры
  - $P$ — сезонная AR
  - $D$ — сезонная разность
  - $Q$ — сезонная MA
  - $s$ — период сезонности (12 для месячных данных с годовой сезонностью)

**Математически:**

$$\Phi_P(L^s) \phi_p(L) \nabla^D_s \nabla^d y_t = \Theta_Q(L^s) \theta_q(L) \varepsilon_t$$

где:
- $\nabla^d$ — обычная разность
- $\nabla^D_s$ — сезонная разность: $y_t - y_{t-s}$

**Пример:** SARIMA(1, 1, 1)(1, 1, 1, 12)
- AR(1), MA(1), differencing=1
- Seasonal AR(1), MA(1), seasonal differencing=1, период=12

---

### 1.6 Prophet (Facebook)

**Prophet** — аддитивная модель:

$$y_t = g(t) + s(t) + h(t) + \varepsilon_t$$

где:
- $g(t)$ — **тренд** (кусочно-линейный или логистический рост)
- $s(t)$ — **сезонность** (Fourier series для годовой/недельной)
- $h(t)$ — **праздники** и события
- $\varepsilon_t$ — шум

**Преимущества Prophet:**
- ✅ Автоматический (мало параметров)
- ✅ Справляется с пропусками
- ✅ Учитывает праздники
- ✅ Интерпретируемые компоненты
- ✅ Робастен к выбросам

**Когда использовать:**
- 📊 Бизнес-метрики (продажи, трафик)
- 🔄 Четкая сезонность (годовая/недельная)
- 🎉 Влияние праздников
- ⏱️ Ежедневные/недельные данные

---

## 💻 Часть 2: Практика

### 2.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Для временных рядов
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Prophet
try:
    from prophet import Prophet
except ImportError:
    print("Prophet not installed. Install: pip install prophet")

# Метрики
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Библиотеки загружены")

### 2.2 Загрузка датасета Airline Passengers

Датасет доступен напрямую через seaborn или statsmodels.

In [ ]:
# Загрузка датасета
try:
    # Через statsmodels
    from statsmodels.datasets import get_rdataset
    airline_data = get_rdataset('AirPassengers', 'datasets')
    df = airline_data.data
    df.columns = ['time', 'passengers']
except:
    # Альтернативный способ
    url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
    df = pd.read_csv(url, parse_dates=['Month'], index_col='Month')
    df.columns = ['passengers']
    df.reset_index(inplace=True)
    df.columns = ['time', 'passengers']

# Преобразуем time в datetime
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

print(f"Датасет загружен: {df.shape[0]} наблюдений")
print(f"Период: {df.index.min()} - {df.index.max()}")
print(f"\nПервые строки:")
df.head()

### 2.3 Exploratory Data Analysis (EDA)

In [ ]:
# Базовая статистика
print("Статистика временного ряда:")
print(df['passengers'].describe())

# График временного ряда
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['passengers'], linewidth=2)
plt.title('Airline Passengers (1949-1960)', fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Number of Passengers', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Наблюдения:")
print("1. ✅ Четкий восходящий тренд")
print("2. ✅ Годовая сезонность (пики летом)")
print("3. ✅ Увеличивающаяся амплитуда сезонности (мультипликативная модель)")
print("4. ❌ Ряд НЕстационарный (требует преобразований)")

### 2.4 Декомпозиция временного ряда

Разложим ряд на **тренд**, **сезонность** и **остатки**.

In [ ]:
# Декомпозиция (мультипликативная модель)
decomposition = seasonal_decompose(df['passengers'], model='multiplicative', period=12)

# Визуализация
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# Исходный ряд
decomposition.observed.plot(ax=axes[0], color='blue')
axes[0].set_title('Original Time Series', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Passengers')

# Тренд
decomposition.trend.plot(ax=axes[1], color='green')
axes[1].set_title('Trend Component', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Trend')

# Сезонность
decomposition.seasonal.plot(ax=axes[2], color='orange')
axes[2].set_title('Seasonal Component (Period=12 months)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Seasonality')

# Остатки
decomposition.resid.plot(ax=axes[3], color='red')
axes[3].set_title('Residuals (Random Noise)', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Residuals')

plt.tight_layout()
plt.show()

print("✅ Декомпозиция показывает:")
print("  - Линейный возрастающий тренд")
print("  - Стабильную годовую сезонность (пики летом)")
print("  - Остатки близки к белому шуму")

### 2.5 Тесты на стационарность

**ADF test:** $H_0$ = ряд нестационарный (есть unit root)  
**KPSS test:** $H_0$ = ряд стационарный

Для стационарности нужно:
- ADF: p-value < 0.05 (отвергаем $H_0$)
- KPSS: p-value > 0.05 (не отвергаем $H_0$)

In [ ]:
def check_stationarity(timeseries, title):
    """Проверка стационарности с ADF и KPSS тестами"""
    print(f"\n{'='*60}")
    print(f"Тест стационарности: {title}")
    print(f"{'='*60}")
    
    # ADF Test
    adf_result = adfuller(timeseries.dropna(), autolag='AIC')
    print(f"\nADF Test (H0: нестационарный):")
    print(f"  ADF Statistic: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Critical values: {adf_result[4]}")
    
    if adf_result[1] < 0.05:
        print(f"  ✅ Ряд СТАЦИОНАРНЫЙ (p < 0.05, отвергаем H0)")
    else:
        print(f"  ❌ Ряд НЕСТАЦИОНАРНЫЙ (p >= 0.05, не отвергаем H0)")
    
    # KPSS Test
    kpss_result = kpss(timeseries.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test (H0: стационарный):")
    print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
    print(f"  p-value: {kpss_result[1]:.4f}")
    print(f"  Critical values: {kpss_result[3]}")
    
    if kpss_result[1] > 0.05:
        print(f"  ✅ Ряд СТАЦИОНАРНЫЙ (p > 0.05, не отвергаем H0)")
    else:
        print(f"  ❌ Ряд НЕСТАЦИОНАРНЫЙ (p <= 0.05, отвергаем H0)")

# Проверка оригинального ряда
check_stationarity(df['passengers'], "Original Series")

### 2.6 Преобразование к стационарности

Применим **log transformation** + **differencing**:

In [ ]:
# Log transformation для стабилизации дисперсии
df['log_passengers'] = np.log(df['passengers'])

# First differencing для устранения тренда
df['log_diff'] = df['log_passengers'].diff()

# Сезонная разность
df['log_seasonal_diff'] = df['log_passengers'].diff(12)

# Визуализация преобразований
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Log transform
df['log_passengers'].plot(ax=axes[0], color='blue')
axes[0].set_title('Log Transformation', fontsize=14, fontweight='bold')
axes[0].set_ylabel('log(Passengers)')

# First difference
df['log_diff'].plot(ax=axes[1], color='green')
axes[1].set_title('Log + First Difference', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Δlog(Passengers)')
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)

# Seasonal difference
df['log_seasonal_diff'].plot(ax=axes[2], color='orange')
axes[2].set_title('Log + Seasonal Difference (lag=12)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Δ12 log(Passengers)')
axes[2].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Проверка стационарности после преобразований
check_stationarity(df['log_diff'], "Log + First Difference")

### 2.7 ACF и PACF графики

**ACF (AutoCorrelation Function):** корреляция с прошлыми значениями  
**PACF (Partial AutoCorrelation Function):** корреляция с учетом промежуточных значений

**Правила:**
- **ACF постепенно затухает, PACF резко обрывается** → AR(p), где p = обрыв на PACF
- **PACF постепенно затухает, ACF резко обрывается** → MA(q), где q = обрыв на ACF
- **Оба затухают** → ARMA(p, q)

In [ ]:
# ACF и PACF графики
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_acf(df['log_diff'].dropna(), lags=40, ax=axes[0])
axes[0].set_title('ACF (AutoCorrelation Function)', fontsize=14, fontweight='bold')

plot_pacf(df['log_diff'].dropna(), lags=40, ax=axes[1])
axes[1].set_title('PACF (Partial AutoCorrelation)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Интерпретация:")
print("  - ACF: значимые лаги на 1, 12, 24 (сезонность)")
print("  - PACF: значимые лаги на 1, 12 (AR компоненты)")
print("  - Предположение: ARIMA(1, 1, 1) + сезонная компонента (12)")

### 2.8 Train/Test Split

**Важно:** Для временных рядов НЕ используем random shuffle!  
Разделяем последовательно: train = первые 80%, test = последние 20%.

In [ ]:
# Разделение на train/test
train_size = int(len(df) * 0.8)

train = df['passengers'][:train_size]
test = df['passengers'][train_size:]

print(f"Train size: {len(train)} ({len(train)/len(df)*100:.1f}%)")
print(f"Test size: {len(test)} ({len(test)/len(df)*100:.1f}%)")
print(f"\nTrain: {train.index.min()} - {train.index.max()}")
print(f"Test: {test.index.min()} - {test.index.max()}")

# Визуализация split
plt.figure(figsize=(14, 5))
plt.plot(train.index, train, label='Train', linewidth=2)
plt.plot(test.index, test, label='Test', linewidth=2, color='orange')
plt.axvline(train.index[-1], color='red', linestyle='--', alpha=0.5, label='Split')
plt.title('Train/Test Split', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Passengers')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 🔮 Часть 3: Модели прогнозирования

### 3.1 ARIMA Model

Используем **ARIMA(p, d, q)** с параметрами из ACF/PACF анализа.  
Попробуем несколько вариантов и выберем лучший по AIC.

In [ ]:
# ARIMA(1, 1, 1) - базовая модель
print("Обучение ARIMA(1, 1, 1)...")
arima_model = ARIMA(train, order=(1, 1, 1))
arima_fitted = arima_model.fit()

print("\n" + "="*60)
print("ARIMA Model Summary")
print("="*60)
print(arima_fitted.summary())

# Прогноз на test
arima_forecast = arima_fitted.forecast(steps=len(test))

# Метрики
arima_mse = mean_squared_error(test, arima_forecast)
arima_rmse = np.sqrt(arima_mse)
arima_mae = mean_absolute_error(test, arima_forecast)

print(f"\n📊 ARIMA Metrics:")
print(f"  RMSE: {arima_rmse:.2f}")
print(f"  MAE: {arima_mae:.2f}")
print(f"  AIC: {arima_fitted.aic:.2f}")
print(f"  BIC: {arima_fitted.bic:.2f}")

In [ ]:
# Визуализация ARIMA predictions
plt.figure(figsize=(14, 6))

plt.plot(train.index, train, label='Train', linewidth=2)
plt.plot(test.index, test, label='Test (Actual)', linewidth=2, color='orange')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast', linewidth=2, 
         color='green', linestyle='--')

plt.axvline(train.index[-1], color='red', linestyle='--', alpha=0.5)
plt.title('ARIMA(1, 1, 1) Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Passengers')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n⚠️ Проблема: ARIMA не улавливает сезонность!")
print("Прогноз линейный, без годовых циклов → нужен SARIMA")

### 3.2 Auto ARIMA - автоматический подбор параметров

Используем `pmdarima` для автоматического поиска лучших параметров:

In [ ]:
try:
    from pmdarima import auto_arima
    
    print("Запуск Auto ARIMA (может занять время)...")
    auto_model = auto_arima(
        train, 
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        d=None,  # автоматический выбор d
        seasonal=False,  # пока без сезонности
        stepwise=True,
        suppress_warnings=True,
        error_action='ignore',
        trace=True
    )
    
    print("\n" + "="*60)
    print("Best Auto ARIMA Model:")
    print("="*60)
    print(auto_model.summary())
    
    # Прогноз
    auto_forecast = auto_model.predict(n_periods=len(test))
    
    # Метрики
    auto_rmse = np.sqrt(mean_squared_error(test, auto_forecast))
    auto_mae = mean_absolute_error(test, auto_forecast)
    
    print(f"\n📊 Auto ARIMA Metrics:")
    print(f"  RMSE: {auto_rmse:.2f}")
    print(f"  MAE: {auto_mae:.2f}")
    
except ImportError:
    print("⚠️ pmdarima не установлен")
    print("Установка: pip install pmdarima")
    auto_forecast = None

### 3.3 SARIMA - Seasonal ARIMA

Добавляем сезонную компоненту: **SARIMA(p, d, q)(P, D, Q, s)**  
Для месячных данных с годовой сезонностью: s=12

In [ ]:
# SARIMA(1, 1, 1)(1, 1, 1, 12)
print("Обучение SARIMA(1, 1, 1)(1, 1, 1, 12)...")
sarima_model = SARIMAX(
    train,
    order=(1, 1, 1),  # ARIMA часть
    seasonal_order=(1, 1, 1, 12),  # Сезонная часть
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_fitted = sarima_model.fit(disp=False)

print("\n" + "="*60)
print("SARIMA Model Summary")
print("="*60)
print(sarima_fitted.summary())

# Прогноз
sarima_forecast = sarima_fitted.forecast(steps=len(test))

# Метрики
sarima_mse = mean_squared_error(test, sarima_forecast)
sarima_rmse = np.sqrt(sarima_mse)
sarima_mae = mean_absolute_error(test, sarima_forecast)

print(f"\n📊 SARIMA Metrics:")
print(f"  RMSE: {sarima_rmse:.2f}")
print(f"  MAE: {sarima_mae:.2f}")
print(f"  AIC: {sarima_fitted.aic:.2f}")
print(f"  BIC: {sarima_fitted.bic:.2f}")

In [ ]:
# Визуализация SARIMA predictions
plt.figure(figsize=(14, 6))

plt.plot(train.index, train, label='Train', linewidth=2)
plt.plot(test.index, test, label='Test (Actual)', linewidth=2, color='orange')
plt.plot(test.index, sarima_forecast, label='SARIMA Forecast', linewidth=2, 
         color='purple', linestyle='--')

plt.axvline(train.index[-1], color='red', linestyle='--', alpha=0.5)
plt.title('SARIMA(1,1,1)(1,1,1,12) Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Passengers')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ SARIMA захватывает сезонность!")
print("Прогноз учитывает годовые циклы (пики летом, спады зимой)")

### 3.4 Prophet (Facebook)

Prophet требует специальный формат данных: columns=['ds', 'y']

In [ ]:
try:
    # Подготовка данных для Prophet
    train_prophet = pd.DataFrame({
        'ds': train.index,
        'y': train.values
    })
    
    # Обучение Prophet
    print("Обучение Prophet...")
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative'  # из-за растущей амплитуды
    )
    
    prophet_model.fit(train_prophet)
    
    # Прогноз
    future = prophet_model.make_future_dataframe(periods=len(test), freq='MS')
    prophet_forecast_full = prophet_model.predict(future)
    
    # Извлекаем прогноз для test периода
    prophet_forecast = prophet_forecast_full['yhat'].iloc[-len(test):].values
    
    # Метрики
    prophet_mse = mean_squared_error(test, prophet_forecast)
    prophet_rmse = np.sqrt(prophet_mse)
    prophet_mae = mean_absolute_error(test, prophet_forecast)
    
    print(f"\n📊 Prophet Metrics:")
    print(f"  RMSE: {prophet_rmse:.2f}")
    print(f"  MAE: {prophet_mae:.2f}")
    
    prophet_available = True
    
except Exception as e:
    print(f"⚠️ Prophet error: {e}")
    print("Установка: pip install prophet")
    prophet_forecast = None
    prophet_available = False

In [ ]:
if prophet_available:
    # Визуализация Prophet
    plt.figure(figsize=(14, 6))
    
    plt.plot(train.index, train, label='Train', linewidth=2)
    plt.plot(test.index, test, label='Test (Actual)', linewidth=2, color='orange')
    plt.plot(test.index, prophet_forecast, label='Prophet Forecast', 
             linewidth=2, color='blue', linestyle='--')
    
    plt.axvline(train.index[-1], color='red', linestyle='--', alpha=0.5)
    plt.title('Prophet Forecast', fontsize=16, fontweight='bold')
    plt.xlabel('Time')
    plt.ylabel('Passengers')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Prophet компоненты
    fig = prophet_model.plot_components(prophet_forecast_full)
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Prophet автоматически разделил:")
    print("  - Тренд (общее направление)")
    print("  - Yearly seasonality (годовая сезонность)")

### 3.5 Сравнение всех моделей

Сравним ARIMA, SARIMA и Prophet по метрикам и визуально.

In [ ]:
# Таблица сравнения
comparison = pd.DataFrame({
    'Model': ['ARIMA(1,1,1)', 'SARIMA(1,1,1)(1,1,1,12)', 'Prophet'],
    'RMSE': [arima_rmse, sarima_rmse, prophet_rmse if prophet_available else None],
    'MAE': [arima_mae, sarima_mae, prophet_mae if prophet_available else None],
    'AIC': [arima_fitted.aic, sarima_fitted.aic, 'N/A'],
    'Сезонность': ['❌', '✅', '✅'],
    'Автоподбор': ['❌', '❌', '✅']
})

print("\n" + "="*80)
print("СРАВНЕНИЕ МОДЕЛЕЙ")
print("="*80)
print(comparison.to_string(index=False))

# Визуальное сравнение
plt.figure(figsize=(14, 7))

plt.plot(train.index, train, label='Train', linewidth=2, alpha=0.7)
plt.plot(test.index, test, label='Test (Actual)', linewidth=3, color='black')
plt.plot(test.index, arima_forecast, label=f'ARIMA (RMSE={arima_rmse:.1f})', 
         linewidth=2, linestyle='--', alpha=0.8)
plt.plot(test.index, sarima_forecast, label=f'SARIMA (RMSE={sarima_rmse:.1f})', 
         linewidth=2, linestyle='--', alpha=0.8)

if prophet_available:
    plt.plot(test.index, prophet_forecast, 
             label=f'Prophet (RMSE={prophet_rmse:.1f})', 
             linewidth=2, linestyle='--', alpha=0.8)

plt.axvline(train.index[-1], color='red', linestyle='--', alpha=0.3, 
            linewidth=2, label='Train/Test Split')
plt.title('Model Comparison: ARIMA vs SARIMA vs Prophet', 
          fontsize=16, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Number of Passengers', fontsize=12)
plt.legend(loc='upper left', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 3.6 Анализ остатков (Residuals Diagnostic)

Хорошая модель должна иметь остатки близкие к **белому шуму**:
- Нормальное распределение
- Нулевое среднее
- Постоянная дисперсия (гомоскедастичность)
- Нет автокорреляции

In [ ]:
# Диагностика остатков SARIMA (лучшая модель)
fig = sarima_fitted.plot_diagnostics(figsize=(14, 10))
plt.tight_layout()
plt.show()

print("\n📊 Интерпретация диагностики остатков:")
print("\n1. Standardized Residuals:")
print("   - Должны быть случайными (без паттернов)")
print("   - Среднее ≈ 0, дисперсия постоянная")
print("\n2. Histogram + KDE:")
print("   - Остатки должны быть нормально распределены (гауссовская кривая)")
print("\n3. Q-Q Plot:")
print("   - Точки должны лежать на диагональной линии")
print("   - Если да → остатки нормальные")
print("\n4. Correlogram (ACF):")
print("   - Все лаги должны быть внутри доверительного интервала (синяя зона)")
print("   - Если да → нет автокорреляции (белый шум)")

## 🎓 Выводы и рекомендации

### 📊 Результаты

**Производительность моделей:**
1. **SARIMA** показала лучшие результаты (низкий RMSE)
   - ✅ Учитывает сезонность
   - ✅ Точнее ARIMA
   - ⚠️ Требует подбора параметров

2. **Prophet** близок к SARIMA
   - ✅ Автоматический (не нужно подбирать p, d, q)
   - ✅ Интерпретируемые компоненты
   - ✅ Легко добавить праздники и внешние регрессоры
   - ⚠️ Может быть менее точным на некоторых данных

3. **ARIMA** без сезонности
   - ❌ НЕ учитывает годовые циклы
   - ❌ Худшая точность
   - ✅ Проще для понимания

---

### 🎯 Когда использовать какую модель?

| Модель | Когда использовать | Когда НЕ использовать |
|--------|-------------------|----------------------|
| **ARIMA** | - Нет четкой сезонности<br>- Короткие ряды<br>- Нужна простота | - Есть сезонность<br>- Нестационарный ряд |
| **SARIMA** | - Четкая сезонность<br>- Средние/длинные ряды<br>- Нужна максимальная точность | - Нет сезонности<br>- Очень длинные ряды (медленно) |
| **Prophet** | - Бизнес-метрики<br>- Праздники важны<br>- Нужна автоматизация<br>- Много пропусков | - Нужен полный контроль<br>- Нет явной сезонности |

---

### 🚀 Следующие шаги: Deep Learning для временных рядов

**Классические методы (ARIMA/SARIMA/Prophet):**
- ✅ Работают отлично на univariate временных рядах
- ✅ Быстрые, интерпретируемые
- ✅ Требуют мало данных
- ❌ Сложно с multivariate (много признаков)
- ❌ Линейные зависимости

**Deep Learning (RNN/LSTM/GRU) - следующий ноутбук:**
- ✅ Нелинейные зависимости
- ✅ Multivariate временные ряды
- ✅ Долгосрочные зависимости (LSTM)
- ✅ Автоматическое извлечение признаков
- ❌ Требуют много данных
- ❌ Сложнее интерпретировать
- ❌ Дольше обучаются

**Рекомендация:**
1. Начинайте с SARIMA/Prophet (быстро, хорошие результаты)
2. Переходите к LSTM, если:
   - Много признаков (multivariate)
   - Нелинейные паттерны
   - Большой датасет (1000+ точек)
   - Нужна экстраполяция на длинные горизонты

---

### 📚 Дополнительные материалы

**Теория:**
- ["Forecasting: Principles and Practice" (Hyndman & Athanasopoulos)](https://otexts.com/fpp3/)
- ["Time Series Analysis" (Hamilton)](https://press.princeton.edu/books/hardcover/9780691042893/time-series-analysis)

**Практика:**
- [statsmodels documentation](https://www.statsmodels.org/stable/tsa.html)
- [Prophet documentation](https://facebook.github.io/prophet/)
- [pmdarima (auto_arima)](http://alkaline-ml.com/pmdarima/)

---

**Phase 3, Step 1 COMPLETE!** ✅  
**Next:** RNN/LSTM/GRU for Time Series (Step 2)